In [150]:
import pandas as pd
import numpy as np
import statsmodels as sm
from statsmodels.tsa.statespace import sarimax
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import time

from typing import List, Dict
from gretel_client import configure_session

In [151]:
# Specify your Gretel API key

pd.set_option("max_colwidth", None)

configure_session(api_key="prompt", cache="yes", validate=True)


Caching Gretel config to disk.
Using endpoint https://api.gretel.cloud
Logged in as hoang.nd@orai.io ✅


In [152]:
# Load timeseries example to a dataframe

data_source = "https://gretel-public-website.s3.amazonaws.com/datasets/credit-timeseries-dataset.csv"
original_df = pd.read_csv(data_source)
original_df.to_csv("original.csv", index=False)
original_df


,net_amt,date,account_balance,credit_amt,district_id,debit_amt
0,50425.0,1995-04-30,485849.7,232833.8,77,182408.8
1,22260.4,1995-03-31,435424.7,269788.6,77,247528.2
2,94953.0,1995-02-28,413164.3,272404.2,77,177451.2
3,13419.3,1994-11-30,476239.2,209552.5,77,196133.2
4,-109497.2,1995-01-31,318211.3,209838.0,77,319335.2
...,...,...,...,...,...,...
5539,320044.0,1996-02-29,997363.3,659139.6,4,339095.6
5540,80647.8,1995-12-31,836115.8,434639.6,4,353991.8
5541,-158796.5,1996-01-31,677319.3,408007.3,4,566803.8
5542,44954.8,1996-03-31,1042318.1,453294.0,4,408339.2


In [153]:
# Gretel Transforms Configuration
config = """
schema_version: "1.0"
models:
    - transforms:
        data_source: "__tmp__"
        policies:
            - name: shiftnumbers
              rules:
                - name: shiftnumbers
                  conditions:
                    field_name:
                        - account_balance
                        - credit_amt
                        - debit_amt
                        - net_amt
                  transforms:
                    - type: numbershift
                      attrs:
                        min: 1
                        max: 100
                        field_name:
                            - date
                            - district_id
"""


In [154]:
# De-identify the original dataset using the policy above
import yaml

from gretel_client.projects import create_or_get_unique_project
from gretel_client.helpers import poll

# Create a project and model configuration.
project = create_or_get_unique_project(name="numbershift-transform")

model = project.create_model_obj(
    model_config=yaml.safe_load(config), data_source=data_source
)

# Upload the training data.  Train the model.
model.submit_cloud()
poll(model)

record_handler = model.create_record_handler_obj(data_source=data_source)
record_handler.submit_cloud()
poll(record_handler)

deid_df = pd.read_csv(record_handler.get_artifact_link("data"), compression="gzip")


INFO: Starting poller


{
    "uid": "633ff763ec698b6d8072488e",
    "guid": "model_2FnmzQ3kLaoLR3ModxbX6lDM7ug",
    "model_name": "enthusiastic-grandiose-wolf",
    "runner_mode": "cloud",
    "user_id": "633ff67bbff621779f449273",
    "user_guid": "user_2FnmWFhqEDJhOMwCmsPcFDdNX3q",
    "billing_domain": null,
    "billing_domain_guid": null,
    "project_id": "633ff74d44a4f15b630a351f",
    "project_guid": "proj_2FnmwfIx7oX7yUa3DKpWSaktQyH",
    "status_history": {
        "created": "2022-10-07T09:54:43.083103Z"
    },
    "last_modified": "2022-10-07T09:54:43.190509Z",
    "status": "created",
    "last_active_hb": null,
    "duration_minutes": null,
    "error_msg": null,
    "error_id": null,
    "traceback": null,
    "annotations": null,
    "container_image": "074762682575.dkr.ecr.us-west-2.amazonaws.com/gretelai/transforms@sha256:c9b59a72bc0f0d73111d7bc7e52a9c5b8cb6eb44734caa23dc02b1672fa7fc51",
    "container_image_version": "2.10.0",
    "model_type": "transform",
    "config": {
        "schema

INFO: Status is created. Model creation has been queued.
INFO: Status is pending. A Gretel Cloud worker is being allocated to begin model creation.
INFO: Status is active. A worker has started creating your model!
2022-10-07T09:54:59.223625Z  Starting transforms model training
2022-10-07T09:54:59.224349Z  Loading training data
2022-10-07T09:54:59.455682Z  Using up to 10000 records for training
2022-10-07T09:54:59.456110Z  Training data loaded
2022-10-07T09:54:59.456607Z  Beginning transforms model training
2022-10-07T09:55:23.642219Z  Saving model archive
2022-10-07T09:55:23.644390Z  Saving training report
2022-10-07T09:55:23.647861Z  Generating data preview with up to 100 records
2022-10-07T09:55:23.719638Z  Done generating data preview
2022-10-07T09:55:23.720824Z  Uploading artifacts to Gretel Cloud
2022-10-07T09:55:24.465169Z  Model creation complete!
INFO: Starting poller


{
    "uid": "633ff79fdae54ecd4ce57618",
    "guid": "model_run_2Fnn6vKNXwnPKkYq4NXS3uJqy0k",
    "model_name": null,
    "runner_mode": "cloud",
    "user_id": "633ff67bbff621779f449273",
    "user_guid": "user_2FnmWFhqEDJhOMwCmsPcFDdNX3q",
    "billing_domain": null,
    "billing_domain_guid": null,
    "project_id": "633ff74d44a4f15b630a351f",
    "project_guid": "proj_2FnmwfIx7oX7yUa3DKpWSaktQyH",
    "status_history": {
        "created": "2022-10-07T09:55:43.516000Z"
    },
    "last_modified": "2022-10-07T09:55:43.648000Z",
    "status": "created",
    "last_active_hb": null,
    "duration_minutes": null,
    "error_msg": null,
    "error_id": null,
    "traceback": null,
    "annotations": null,
    "container_image": "074762682575.dkr.ecr.us-west-2.amazonaws.com/gretelai/transforms@sha256:c9b59a72bc0f0d73111d7bc7e52a9c5b8cb6eb44734caa23dc02b1672fa7fc51",
    "container_image_version": "2.10.0",
    "model_id": "633ff763ec698b6d8072488e",
    "model_guid": "model_2FnmzQ3kLaoLR3

INFO: Status is pending. A Gretel Cloud worker is being allocated to begin transforming records.
INFO: Status is active. A worker has started!
2022-10-07T09:55:58.913042Z  Downloading model to worker
2022-10-07T09:56:03.582976Z  Loading model
2022-10-07T09:56:03.898386Z  Loading dataset to transform
2022-10-07T09:56:04.313508Z  Starting transform
2022-10-07T09:56:06.159915Z  Transform in progress
{
    "records_transformed": 5544,
    "new_records_transformed": 5544
}
2022-10-07T09:56:06.160460Z  Dataset transform complete
2022-10-07T09:56:06.398320Z  Uploading data to Gretel Cloud
2022-10-07T09:56:06.668961Z  Uploading run report to Gretel Cloud


In [155]:
# View the transformation report

import json
from smart_open import open

report = json.loads(open(model.get_artifact_link("report_json")).read())
pd.DataFrame(report["metadata"]["fields"])


,name,count,approx_distinct_count,missing_count,labels,attributes,entities,types
0,net_amt,5544,5453,0,[],[],[],"[{'type': 'number', 'count': 5544}]"
1,date,5544,72,0,[datetime],[],"[{'label': 'datetime', 'count': 5544, 'approx_distinct_count': 72, 'f_ratio': 1.0, 'sources': ['gretel/datetime']}]","[{'type': 'string', 'count': 5544}]"
2,account_balance,5544,5465,0,[],[],[],"[{'type': 'number', 'count': 5544}]"
3,credit_amt,5544,5466,0,[],[],[],"[{'type': 'number', 'count': 5544}]"
4,district_id,5544,77,0,[],[],"[{'label': 'location', 'count': 4896, 'approx_distinct_count': 68, 'f_ratio': 0.8831168831168831, 'sources': ['gretel/location']}]","[{'type': 'number', 'count': 5544}]"
5,debit_amt,5544,5319,0,[],[],[],"[{'type': 'number', 'count': 5544}]"


In [156]:
# Here we sort and remove "net_amt" as it's a derived column,
# We will add back in after the data is synthesized
train_df = deid_df.copy()

train_df.sort_values("date", inplace=True)
train_cols = list(train_df.columns)
train_cols.remove("net_amt")
train_df = train_df.filter(train_cols)

# Here we noticed that some number have extremely long precision,
# so we round the data
train_df = train_df.round(1)
train_df.to_csv("train.csv", index=False)
train_df


,date,account_balance,credit_amt,district_id,debit_amt
1814,1993-01-31,1132.0,1132.0,52,32.0
5392,1993-01-31,91.0,91.0,2,91.0
3888,1993-01-31,264.0,264.0,23,64.0
1509,1993-01-31,30.0,30.0,56,30.0
1106,1993-01-31,6010.5,6010.5,62,66.0
...,...,...,...,...,...
5086,1998-12-31,2044862.7,827593.8,3,692353.7
169,1998-12-31,5873315.7,3070591.4,74,2491755.5
3228,1998-12-31,1599165.1,525864.4,30,462125.0
4743,1998-12-31,2266274.9,1086885.0,9,1010709.2


In [157]:
from gretel_client.projects.models import read_model_config

# Create a project and model configuration.
project = create_or_get_unique_project(name="ts-5544-regular-seed")

# Pull down the default synthetic config.  We will modify it slightly.
config = read_model_config("synthetics/default")

# Set up the seed fields
seed_fields = ["date", "district_id"]

task = {
    "type": "seed",
    "attrs": {
        "fields": seed_fields,
    },
}

# Fine tune model parameters. These are the parameters we found to work best.  This is "Run 20" in the document
config["models"][0]["synthetics"]["task"] = task

config["models"][0]["synthetics"]["params"]["vocab_size"] = 20
config["models"][0]["synthetics"]["params"]["learning_rate"] = 0.005
config["models"][0]["synthetics"]["params"]["epochs"] = 100
config["models"][0]["synthetics"]["params"]["gen_temp"] = 0.8
config["models"][0]["synthetics"]["params"]["reset_states"] = True
config["models"][0]["synthetics"]["params"]["dropout_rate"] = 0.5
config["models"][0]["synthetics"]["params"]["gen_temp"] = 0.8
config["models"][0]["synthetics"]["params"]["early_stopping"] = True
config["models"][0]["synthetics"]["privacy_filters"]["similarity"] = None
config["models"][0]["synthetics"]["privacy_filters"]["outliers"] = None
config["models"][0]["synthetics"]["generate"]["num_records"] = train_df.shape[0]

# Get a csv to work with, just dump out the train_df.
deid_df.to_csv("train.csv", index=False)

# Initiate a new model with the chosen config
model = project.create_model_obj(model_config=config, data_source="train.csv")

# Upload the training data.  Train the model.
model.submit_cloud()
poll(model)

synthetic = pd.read_csv(model.get_artifact_link("data_preview"), compression="gzip")
synthetic


INFO: Starting poller


{
    "uid": "633ff7cfec698b6d8072488f",
    "guid": "model_2FnnCwZ81X1UFQJm3jxulpXFjGw",
    "model_name": "default-config",
    "runner_mode": "cloud",
    "user_id": "633ff67bbff621779f449273",
    "user_guid": "user_2FnmWFhqEDJhOMwCmsPcFDdNX3q",
    "billing_domain": null,
    "billing_domain_guid": null,
    "project_id": "633ff7cb44a4f15b630a3521",
    "project_guid": "proj_2FnnCQr0OEdh3qjU6sCziPjona1",
    "status_history": {
        "created": "2022-10-07T09:56:31.730375Z"
    },
    "last_modified": "2022-10-07T09:56:31.852629Z",
    "status": "created",
    "last_active_hb": null,
    "duration_minutes": null,
    "error_msg": null,
    "error_id": null,
    "traceback": null,
    "annotations": null,
    "container_image": "074762682575.dkr.ecr.us-west-2.amazonaws.com/gretelai/synthetics@sha256:3339cda15d1ac57a84a12275267fe2f59cf27b43e00b916e8f4f154b5e8398a1",
    "container_image_version": "2.10.0",
    "model_type": "synthetics",
    "config": {
        "schema_version": "

INFO: Status is created. Model creation has been queued.
INFO: Status is pending. A Gretel Cloud worker is being allocated to begin model creation.


In [ ]:
# Add back in the derived column "net_amt"
net_amt = synthetic["credit_amt"] - synthetic["debit_amt"]
synthetic["net_amt"] = net_amt

# Save off the new synthetic data
synthetic.to_csv("synthetic.csv", index=False, header=True)


In [ ]:
# View the Synthetic Performance Report
import IPython
from smart_open import open

IPython.display.HTML(data=open(model.get_artifact_link("report")).read(), metadata=dict(isolated=True))


In [ ]:
import matplotlib
import matplotlib.pyplot as plt


def plot_district_averages(
    synthetic: pd.DataFrame, training: pd.DataFrame, district_id: int
) -> pd.DataFrame:

    synthetic_data = synthetic.loc[synthetic["district_id"] == district_id]
    synthetic_data = synthetic_data.set_index("date")

    training_data = training.loc[training["district_id"] == district_id]
    training_data = training_data.set_index("date")

    combined = synthetic_data.join(
        training_data, lsuffix="_synthetic", rsuffix="_original"
    )
    plt.suptitle("District #" + str(district_id))

    for col in ["credit_amt", "debit_amt", "account_balance"]:
        fig = combined.plot(y=[f"{col}_synthetic", f"{col}_original"], figsize=(12, 8))
        plt.title("Time Series for District #" + str(district_id))

    return combined


combined = plot_district_averages(synthetic, train_df, 13)


In [ ]:
import warnings

warnings.filterwarnings("ignore")


def ARIMA_run(
    data_paths: List[str],
    targets: List[str] = None,
    entity_column: str = "district_id",
    entities: List = None,
    date_column: str = "date",
    date_threshold: str = None,
) -> Dict[str, List[float]]:
    """
    Purpose of this function is to automate the run and scoring of SARIMAX models, so we can benchmark results against various different synthetic data configurations.
    The data paths from s3 are passed in, and then entire run, from loading in and sorting the data to creating a model and scoring it, is done via this function.
    The outputs are the target scores for each variable on each dataset's model. This gets used to create bar charts of the RMSE.
    With some fine tuning, this function can be made as a general purpose SARIMAX benchmark function for a variety of datasets.

    Args:
      data_paths: a list of paths to the data you want to create models and score with. These can be either local paths or ones from public buckets.
      targets: Which columns in the data will be your target variables?
      entity_column: This is purely used for datasets that have multiple time series data points from multiple places. Since this function was built with that in mind, it assumes that you will
      give a column that denotes those different places/entities. If None is provided, no handler has been built yet that can handle that.
      entities: This should be a list of the set of entities within the entity column.
      date_column: This should be something we can use to sort the data, so that the time series is read appropriately.
      date_threshold: This is to split the data into train and test. Whatever date you want to threshold by to make the train and test should be specified here.

    Outputs:
      target_scores: This will be a dictionary of RMSE scores for each target variable on each synthetic dataset.
    """
    target_scores = {}
    for target in targets:
        target_scores[target] = []
    for path in data_paths:
        sorted_data = pd.read_csv(path)
        sorted_data.sort_values(date_column, inplace=True)
        sorted_data.drop_duplicates(subset=[date_column, entity_column], inplace=True)

        print("Path: {}".format(path))
        for entity in entities:
            print("Entity: {}".format(entity))
            for target in targets:
                train_data = sorted_data[sorted_data[entity_column] == entity][
                    sorted_data[date_column] < date_threshold
                ]
                test_data = sorted_data[sorted_data[entity_column] == entity][
                    sorted_data[date_column] >= date_threshold
                ]

                model = sarimax.SARIMAX(
                    train_data[target], order=(0, 1, 1), seasonal_order=(1, 1, 0, 12)
                )
                res = model.fit()

                preds = res.forecast(len(test_data[target]))
                rmse = mean_squared_error(test_data[target], preds, squared=False)
                target_scores[target].append(rmse)
                print("Target: {}".format(target))
                print("RMSE: {}".format(rmse))

    return target_scores


In [ ]:
target_scores = ARIMA_run(
    ["synthetic.csv", "original.csv"],
    targets=["net_amt", "account_balance", "credit_amt", "debit_amt"],
    entities=[13],
    date_threshold="1998-01-01",
)
target_scores


In [ ]:
import plotly.express as px

results = pd.DataFrame.from_dict(target_scores)
results["method"] = ["synthetic", "real world"]
results.plot.bar(x="method", title="RMSE per field and run in synthetic timeseries")
